
## Spatial Analysis of Dog Ownership in Zurich: A Spatiotemporal Data Science Approach

### Introduction

#### Problem Statement:
Can we develop a data-driven model by the end of January 2024 that predicts what the *dog puppulation density* will be this year across Zurich’s 34 neighborhoods, with a Mean Absolute Error of less than 10%, using time series cross validation, thereby providing valuable insights for urban planning, pet-related businesses, and community welfare?


#### Context:
As Zurich embarks on a new era post-2017, sans mandatory dog training courses, the city is undertaking a comprehensive exploration of *dog puppulation density* dynamics within its neighborhoods. This initiative, prompted by regulatory shifts, aims to sniff out patterns in *dog puppulation density* that impact urban planning, business ventures, and the overall welfare of our four-legged friends and their humans. The study leverages data from 2015 to 2020 to optimize urban planning, enhance pet-related business ventures, and promote community welfare through an informed understanding of *dog puppulation density* patterns. This tail-wagging study is crucial in this new era for Zurich, providing actionable insights for the immediate future. The goal is to develop a data-driven model that accurately predicts the *dog puppulation density* across Zurich’s 34 neighborhoods in the immediate future.

#### Criteria for Success:
Our goal is to *dig up* clear patterns of *dog puppulation density* in Zurich’s neighborhoods, laying the groundwork for informed future predictions. We aim to *unleash* the potential of our predictive models, forecasting 2024 *dog puppulation density* patterns in Zurich with a Mean Absolute Error of less than 10%. Achieving this would be a *pawsitive* step towards informed future urban strategies


#### Constraints within Solution Space:
- **Temporal Scope**: The study is confined to the years with full data availability across all datasets (2015-2020)
- **Spatial Resolution**: The study focuses on *dog puppulation density* at the neighborhood level. This may not capture variations within neighborhoods or between smaller areas.
- **Generalizability**: The findings of this study are specific to Zurich and may not be applicable to other cities or regions with different demographic, economic, and cultural contexts.


#### Stakeholders:
- **City Planners and Local Authorities:** Empower data-driven decision-making to enhance urban living conditions.
- **Business Enterprises:** Guide service offerings and marketing strategies.
- **Dog Owners:** Offer insights into community resources and pet care options.


#### Key Data Sources:
- **Geospatial Boundaries:** [Zurich Statistical Quarters](https://data.stadt-zuerich.ch/dataset/geo_statistische_quartiere)
- **Dog Ownership Records:** [Dog Owners Dataset](https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001/download/KUL100OD1001.csv)
- **Demographic Statistics:** [Population Dataset](https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903/download/BEV390OD3903.csv)
- **Economic Indicators:** [Income Dataset](https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003/download/WIR100OD1003.csv)
- **Household Dynamics:** [Household Size Dataset](https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806/download/BEV380OD3806.csv)

#### Analytical Objectives:
**Understand the Relationship**: Dig into the relationship between demographic factors and *dog puppulation density* across Zurich’s neighborhoods.
**Identify Trends and Clusters**: Track and map out the spatial and temporal trends of *dog puppulation density*. Identify spatial clusters of high and low *dog puppulation density*.
**Predict Future Trends**: Predict the near-future trends of *dog puppulation density* using historical data, aiming for a Mean Absolute Error of less than 10%. This includes forecasting where Zurich’s dog ‘puppulation’ will be booming across its 34 neighborhoods in the immediate future.


### Imports & Configurations

This section includes the necessary imports for libraries, configuration settings for dataframes and visualizations. These components establish the foundational setup for subsequent data analysis and exploration. 


In [ ]:
# Standard libraries for data manipulation and numerical operations
from IPython.display import clear_output
from PIL import ImageDraw, Image  # For image processing
from urllib.request import urlopen
import math
import pandas as pd
import numpy as np

# Visualization and spatial analysis libraries
from matplotlib import pyplot as plt
import seaborn as sns
import geopandas as gpd
import geoviews as gv
import hvplot.pandas  # noqa
import holoviews as hv
from holoviews import streams
import cartopy.crs as ccrs  # For cartographic projections and geographic plots
import panel as pn
import panel.widgets as pnw
import colorcet as cc  # Additional color palettes

# Libraries for specific functionalities
from esda.moran import Moran, Moran_Local  # Spatial autocorrelation statistics
from fiona.io import ZipMemoryFile
from splot.esda import plot_local_autocorrelation
from thefuzz import fuzz  # For string matching
from wordcloud import WordCloud  # For generating word cloud visualizations
import libpysal as lps  # Spatial analysis library

# Custom helper modules
from translate_app import translate_list_to_dict
import helper_functions as hf  # Custom helper functions for this project

# Additional configurations for visualization libraries
gv.extension("bokeh")
hv.extension("bokeh")
hvplot.extension("bokeh")
pn.extension(sizing_mode="stretch_width", max_width=1000)
clear_output()

In [ ]:
# # Standard libraries for data manipulation and numerical operations
# import math
# import numpy as np
# import pandas as pd

# # Visualization and spatial analysis libraries
# from matplotlib import pyplot as plt
# import seaborn as sns
# import geopandas as gpd
# import geoviews as gv
# import hvplot.pandas  # Enhanced pandas integration with HoloViews
# import holoviews as hv
# from holoviews import streams
# import cartopy.crs as ccrs  # For cartographic projections and geographic plots
# import panel as pn
# import panel.widgets as pnw
# import colorcet as cc  # Additional color palettes

# # Libraries for specific functionalities
# from thefuzz import fuzz  # For string matching
# from wordcloud import WordCloud  # For generating word cloud visualizations

# from IPython.display import clear_output

# from PIL import ImageDraw, Image  # For image processing
# from urllib.request import urlopen
# from fiona.io import ZipMemoryFile
# from esda.moran import Moran, Moran_Local  # Spatial autocorrelation statistics
# from splot.esda import plot_local_autocorrelation
# import libpysal as lps  # Spatial analysis library

# # Custom helper modules
# # Custom function for translation
# from translate_app import translate_list_to_dict
# import helper_functions as hf  # Custom helper functions for this project

# # Additional configurations for visualization libraries
# gv.extension("bokeh")
# hv.extension("bokeh")
# hvplot.extension("bokeh")
# pn.extension(sizing_mode="stretch_width", max_width=1000)
# clear_output()

In [ ]:
# Pandas display options
# Disable warnings for chained assignments
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

# Seaborn style setting
sns.set_style("whitegrid")

# Panel configuration for improved interactivity performance
pn.config.throttled = True

# Clear any output created by the extensions and settings
clear_output()

In [ ]:
# # suppress warnings
# pd.options.mode.chained_assignment = None  # default='warn'
# pd.options.display.max_columns = 50
# pd.options.display.max_rows = 100
# gv.extension("bokeh")
# hv.extension("bokeh")
# hvplot.extension("bokeh")
# pn.extension(sizing_mode="stretch_width", max_width=1000)

# pn.config.throttled = True
# sns.set_style("whitegrid")

# clear_output()  # clear the blank lines output from holoviews

### Data Description

Transformation: Column names were first translated to English using a custom function (`hf.sanitize_df_column_names`) and further refined manually for clarity.

[Zurich Dogs Dataset](#zurich-dog-dataset)<br>
In this dataset you will find information on dogs and their owners from the owners of the municipal dog register since 2015. In the case of dog owners, information on the age group, gender and statistical neighbourhood of the place of residence is provided. For each dog, the breed, the breed type, the sex, the year of birth, the age and the color is recorded. The dog register is maintained by the Dog Control Department of the Zurich City Police. According to the law on the keeping of dogs, the city police are obliged to keep a register of dogs kept in the city of Zurich. Every dog over the age of three months must be registered at the dog control by the owner in person or by means of the registration form.<br>
[Source](https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001)

[Zurich Population Dataset](#Zurich-Population-Dataset)<br>
Number of economic resident population by statistical urban district, origin, sex, age year, 5-year age groups and year, since 1993.<br>
[Source](https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903)

[Zurich Income Dataset](#zurich-income-data)<br>
These data contain quantile values of the taxable income of natural persons who are primarily taxable in the city of Zurich.<br>
[Source](https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003)

[Zurich Household Dataset](#zurich-household-dataset)<br>
Number of private households as well as the economic population of the city of Zurich in private households by household size, urban district, statistical urban district and year, since 2013. A household includes all persons who live together in the same apartment. For this purpose, the term "economic residence" is used for persons. Collective households (homes, hospitals, penal institutions, communal accommodation for asylum seekers, etc.) are not taken into account.<br>
[Source](https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806)



<!-- 
[Zurich Statistical Districts Geospatial Data](#zurich-statistical-districts-geospatial-data)<br>
Zurich's 34 statistical districts, derived from 12 districts and 22 urban districts, form the basis for spatial differentiation in statistical analysis. These divisions are pivotal for administrative tasks and urban planning within the City of Zurich.<br>[Source](https://data.stadt-zuerich.ch/dataset/geo_statistische_quartiere)


[Zurich Dogs Dataset](#zurich-dog-dataset) ||[Source](https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001)<br>
In this dataset you will find information on dogs and their owners from the owners of the municipal dog register since 2015. In the case of dog owners, information on the age group, gender and statistical neighbourhood of the place of residence is provided. For each dog, the breed, the breed type, the sex, the year of birth, the age and the color is recorded. The dog register is maintained by the Dog Control Department of the Zurich City Police.

According to the law on the keeping of dogs, the city police are obliged to keep a register of dogs kept in the city of Zurich. Every dog over the age of three months must be registered at the dog control by the owner in person or by means of the registration form.


[Zurich Population Dataset](#Zurich-Population-Dataset) || [Source](https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903)<br>
Number of economic resident population by statistical urban district, origin, sex, age year, 5-year age groups and year, since 1993.


[Zurich Income Dataset](#zurich-income-data) || [Source](https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003)<br>
These data contain quantile values of the taxable income of natural persons who are primarily taxable in the city of Zurich.


[Zurich Household Dataset](#zurich-household-dataset) || [Source](https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806)<br>
Number of private households as well as the economic population of the city of Zurich in private households by household size, urban district, statistical urban district and year, since 2013.

A household includes all persons who live together in the same apartment. For this purpose, the term "economic residence" is used for persons. Collective households (homes, hospitals, penal institutions, communal accommodation for asylum seekers, etc.) are not taken into account. -->

### Data Preprocessing

##### Column Name Transformation
To enhance readability and ensure consistency across datasets, original column names were translated from German to English and standardized to snake case using our `sanitize_df_column_names` helper function. This transformation facilitates a cleaner, more uniform `pd.DataFrame` structure for analysis.


#### Zurich Statistical Districts Geospatial Data



The geospatial data is organized systematically for enhanced clarity and analysis efficiency:
- `z_gdf_0`: Point data for optimal label positioning on maps.
- `z_gdf_1`: Data tailored for visual representation in cartography.
- `z_gdf_2`: Data designed for area calculations and spatial intersections.
This structured approach ensures effective utilization of geospatial information in our analysis.


In [ ]:
zip_gdf_url = "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_statistical_quarters.zip"
zurich_geo_dicts = hf.get_gdf_from_zip_url(zip_gdf_url)
zurich_desc_path = "../data/zurich_districts.csv"
# small dataframe with a description of each district
zurich_desc = pd.read_csv(zurich_desc_path).drop(columns=["desc_length"])
zurich_desc["link"] = zurich_desc["district"].apply(
    lambda x: f"https://www.zuerich.com/en/visit/about-zurich/zurichs-districts#s-{x}"
)
z_gdf = hf.rename_keys(zurich_geo_dicts, prefix="z_gdf_")

In [ ]:
zurich_desc

In [ ]:
# Look at the data for each of the geodataframes
z_gdf["z_gdf_0"].info()
z_gdf["z_gdf_0"].sample(3)

In [ ]:
z_gdf["z_gdf_1"].info()
z_gdf["z_gdf_1"].sample(3)

In [ ]:
z_gdf["z_gdf_2"].info()
z_gdf["z_gdf_2"].sample(3)


The Zurich geospatial data has been transformed for improved clarity and usability. Key changes include:
- **Column Renaming:** `qname` to `neighborhood`, `qnr` to `sub_district`, `knr` to `district`.
- **Data Type Adjustments:** `sub_district` formatted as a string with leading zeros.
- **Column Selection:** Refined `neighborhood_gdf` with key columns: `neighborhood`, `sub_district`, `district`, `geometry`.
- **Coordinate Reference System (CRS):** Set to WGS 84 (EPSG:4326) for accurate geodetic latitude and longitude.

The result is a streamlined `neighborhood_gdf`, offering a clear and structured representation of Zurich's districts for further spatial analysis.


In [ ]:
zurich_map_gdf = z_gdf["z_gdf_1"]

zurich_map_gdf.rename(
    columns={"qname": "neighborhood",
             "qnr": "sub_district", "knr": "district"},
    inplace=True,
)
zurich_map_gdf["sub_district"] = zurich_map_gdf["sub_district"].astype(
    str).str.zfill(3)

# Create the refined geodataframe
neighborhood_gdf = zurich_map_gdf[
    ["neighborhood", "sub_district", "district", "geometry"]
].copy()

# Display geodataframe information and CRS
neighborhood_gdf.info()
display(neighborhood_gdf.crs)

# Display a sample entry from the transformed geodataframe
neighborhood_gdf.sample().T

In [ ]:
# Load the geospatial data for calculation
zurich_calc_gdf = z_gdf["z_gdf_2"]

# Calculate area in square meters and add as a new column
zurich_calc_gdf["area_km2"] = zurich_calc_gdf.to_crs(
    ccrs.GOOGLE_MERCATOR).area / 1e6

# Rename the column for consistency with the main geodataframe
zurich_calc_gdf = zurich_calc_gdf.rename(columns={"qname": "neighborhood"})

# Merge calculated features with the main geodataframe (neighborhood_gdf)
merged_gdf = neighborhood_gdf.merge(
    zurich_calc_gdf[["neighborhood", "area_km2"]], on="neighborhood"
)

# Display a snapshot of the merged geodataframe
print(merged_gdf[["neighborhood", "area_km2"]].head())

In [ ]:
merged_gdf.to_file("../data/zurich_neighborhoods.geojson")
# merged_gdf

In [ ]:
gpd.read_file("../data/zurich_neighborhoods.geojson").info()
# merged_gdf.info()

In [ ]:
poly_opts = dict(
    width=500,
    height=500,
    color_index=None,
    xaxis=None,
    yaxis=None,
    backend_opts={"toolbar.autohide": True},
)
# Neighborhood polygons
neighborhood_poly = gv.Polygons(merged_gdf).opts(
    tools=["hover", "tap"],
    **poly_opts,
    line_color="skyblue",
    line_width=2,
    fill_color="lightgray",
    fill_alpha=0,
)
# add a basemap
basemap = gv.tile_sources.CartoLight().opts(alpha=0.5, bgcolor="black")

In [ ]:
districts_gdf = (
    neighborhood_gdf.drop(columns=["neighborhood", "sub_district"])
    .dissolve(by="district")
    .reset_index()
)
districts_gdf = districts_gdf.dissolve(by="district").reset_index()
# District polygons
districts_poly = gv.Polygons(districts_gdf.merge(zurich_desc)).opts(
    **poly_opts,
    line_color="pink",
    fill_alpha=0.02,
    tools=["tap", "box_select"],
    line_width=3
)

district_neighborhoods = (basemap * neighborhood_poly * districts_poly).opts(
    title="Zurich Districts and Neighborhoods"
)
pn.state.kill_all_servers()

pn.pane.HoloViews(district_neighborhoods)

The district descriptions are visualized through a word cloud, shaped like the respective districts. Clicking on a district generates its word cloud, alongside a pane displaying the description and the provided source. This creative representation offers an engaging way to explore a district's characteristics.


In [ ]:
# create a stream which selects a district from the map
select_district = streams.Selection1D(source=districts_poly)


@pn.depends(select_district.param.index)
def display_info(index):
    """Displays a brief description of the selected district"""
    if not index:
        return pn.pane.Markdown("No district selected")
    else:
        selected_district = (
            districts_poly.iloc[index[0]]
            .data[["district", "district_name", "desc", "link"]]
            .drop_duplicates()
        )
        dname = selected_district["district_name"].values[0]
        dnum = selected_district["district"].values[0]
        ddesc = selected_district["desc"].values[0]
        link = selected_district["link"].values[0]
        return pn.pane.Markdown(
            f"""
            <div style="
            border: 2px solid #4a4a4a;
            border-radius: 10px;
            padding: 20px 20px 20px 20px;
            background-color: #f9f9f9;
            box-shadow: 0 4px 8px 0 rgba(0,0,0,0.2);
            word-wrap: break-word;
            ">
            <h2 style='color: #008080;'>{dnum}</h2>
            <h1 style='color: #000080;'>{dname}</h1>
            <h3 style='color: #708090;'>{ddesc}</h3>
            <a href="{link}" >Source</a>
            </div>
            
            """,
            width=300,
        )


@pn.depends(select_district.param.index)
def display_wordcloud(index):
    """Displays a wordcloud of the selected district based on the description
    of the district in the shape of the district polygon"""
    if len(index) == 0:
        text = "district select on map"
        wordcloud = WordCloud(width=800, height=500, background_color="white").generate(
            text
        )
        return hv.RGB(np.array(wordcloud)).opts(
            width=500, height=500, active_tools=["box_zoom"]
        )
    else:
        selected_district = (
            districts_poly.iloc[index[0]]
            .data[["district", "district_name", "desc"]]
            .drop_duplicates()
        )
        dname = selected_district["district_name"].values[0]
        dnum = selected_district["district"].values[0]
        ddesc = selected_district["desc"].values[0]
        text = f"{dnum} {dname} {ddesc}"

        polygon = districts_poly.iloc[index[0]].data["geometry"].iloc[0]

        # Get the bounding box of the polygon
        minx, miny, maxx, maxy = polygon.bounds

        # Calculate the width and height of the bounding box
        margin = 0.1
        width = (maxx - minx) * (1 + margin)
        height = (maxy - miny) * (1 + margin)
        # Calculate the new minimum x and y coordinates
        minx -= width * margin / 2
        miny -= height * margin / 2

        # Create a new image with the same aspect ratio as the bounding box
        image_width = 800
        image_height = int(image_width * height / width)
        test = Image.new("1", (image_width, image_height), 0)

        # Convert the coordinates to a numpy array
        coords = np.array(list(polygon.exterior.coords))
        coords -= [minx, miny]
        coords *= [image_width / width, image_height / height]
        coords[:, 1] = image_height - coords[:, 1]
        # Convert the coordinates back to a list of tuples
        scaled_coords = list(map(tuple, coords))

        # Draw the scaled polygon onto the image
        ImageDraw.Draw(test).polygon(scaled_coords, outline=1, fill=1)

        wordcloud = WordCloud(
            mask=~np.array(test) * 255,
            # color_func=lambda *args, **kwargs: breed_color,
            include_numbers=True,
            margin=20,
            # contour_color=breed_color,
            contour_width=5,
            width=800,
            height=500,
            background_color="white",
        ).generate(text)
        return hv.RGB(np.array(wordcloud)).opts(
            width=800,
            height=500,
            tools=["box_zoom"],
            active_tools=["box_zoom"],
            xaxis=None,
            yaxis=None,
            backend_opts={"toolbar.autohide": True}
            # toolbar=None,
        )


district_layout = pn.Column(
    pn.pane.HoloViews(display_wordcloud),
    # pn.bind(display_wordcloud, select_district.param.index),
    pn.Row(
        districts_poly * neighborhood_poly,
        pn.panel(display_info, width=300),
        # pn.bind(display_info, select_district.param.index),
    ),
    sizing_mode="stretch_width",
)

district_layout_card = pn.Card(
    district_layout,
    title="District Descript",
    sizing_mode="stretch_width",
)
district_layout_card

#### Zurich Dogs Dataset

In [ ]:
zurich_dog_data_link = "https://data.stadt-zuerich.ch/dataset/sid_stapo_hundebestand_od1001/download/KUL100OD1001.csv"
zurich_dog_data_link = (
    "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_dogs.csv"
)
zurich_dog_data = pd.read_csv(zurich_dog_data_link)
zurich_dog_data.info()
zurich_dog_data.columns

In [ ]:
zurich_dog_data = hf.sanitize_df_column_names(zurich_dog_data)
zurich_dog_data.info()
zurich_dog_data.sample(3)
zurich_dog_data.columns


This data set includes many columns, some of which are redundant. Out of the 31 columns, we have selected the following 18 for our analysis.

 - deadline_date_year 
 - holder_id 
 - age_v_10_cd 
 - sex_cd 
 - circle_cd 
 - quar_cd 
 - quar_lang
 - race_1_text
 - race_2_text
 - breed_mixed__breed_cd 
 - breed_mongrel_long
 - breed_mixed__breed_sort 
 - breed_type_cd
 - birth_dog_year 
 - age_v_dog_cd 
 - sex_dog_cd 
 - dog_color_text
 - number_of_dogs


##### Data Transformation and Feature Engineering

Before diving into the transformation process, let's explore some key aspects of the Zurich Dogs Dataset to understand its structure and characteristics.


In [ ]:
zurich_dog_data_cols_to_keep = [
    "deadline_date_year",
    "holder_id",
    "age_v_10_cd",
    "sex_cd",
    "circle_cd",
    "quar_cd",
    "quar_lang",
    "race_1_text",
    "race_2_text",
    "breed_mixed__breed_cd",
    "breed_mongrel_long",
    "breed_mixed__breed_sort",
    "breed_type_cd",
    "birth_dog_year",
    "age_v_dog_cd",
    "sex_dog_cd",
    "dog_color_text",
    "number_of_dogs",
]
zurich_dog_data = zurich_dog_data[zurich_dog_data_cols_to_keep]
zurich_dog_data_descriptive_stats = zurich_dog_data.describe(include="all")

zurich_dog_data_descriptive_stats.T.sort_values(by="unique").fillna("")

A few things stand out here. Looking at the `dog_age` we have an extremely low minimum value which is most likely due to error input we also have an extremely high max value which may be also an error. 
the `district` Value also has a high Max number but that may be due to a placeholder for unknown values as we know we only have 12 districts. For categorical values we can see that the `breed_type` also contains some extra values. Only have three breed types instead of 4. finally for the `neighborhood` or `sub_district` columns we know that we only have 34 neighborhoods so therefore the value of 40 includes some extra values which need to be corrected

In [ ]:
# Define a list of conditions and corresponding columns to be updated
conditions = [
    (zurich_dog_data["age_v_10_cd"] > 200, "age_v_10_cd"),
    (zurich_dog_data["circle_cd"] > 12, "circle_cd"),
    (zurich_dog_data["birth_dog_year"] > 2100, "birth_dog_year"),
    (zurich_dog_data["age_v_dog_cd"] > 30, "age_v_dog_cd"),
    (zurich_dog_data["breed_type_cd"] == "UN", "breed_type_cd"),
    (
        (zurich_dog_data["quar_cd"] % 10 == 0) | (
            zurich_dog_data["quar_cd"] > 200),
        "quar_cd",
    ),
    (
        (zurich_dog_data["quar_cd"] % 10 == 0) | (
            zurich_dog_data["quar_cd"] > 200),
        "quar_lang",
    ),
]

# Iterate over the conditions and columns and update the DataFrame
for condition, column in conditions:
    if column == "breed_type_cd":
        zurich_dog_data.loc[condition, column] = "K"
    elif column == "quar_lang":
        zurich_dog_data.loc[condition, column] = ""
    else:
        zurich_dog_data.loc[condition, column] = np.nan

# Check the number of NaN values in each column
zurich_dog_data.isna().sum()

In [ ]:
# Drop the columns with less than 20 nans
zurich_dog_data = zurich_dog_data.dropna(
    subset=["birth_dog_year", "quar_cd", "circle_cd"]
)

##### Breed and Mixed Type Transformation

In this step, we analyze and transform columns related to dog breeds and mixed types. This includes handling mixed breed categories and creating additional features for analysis.


In [ ]:
# Unique values for "breed_mongrel_long"
breed_cat_list_de = zurich_dog_data["breed_mongrel_long"].unique().tolist()
print("Breed Categories (German):")
pprint(breed_cat_list_de, indent=4)

# Create a dictionary for translation
breed_cat_dict = translate_list_to_dict(breed_cat_list_de)
print("\nBreed Category Dictionary (Translation):")
pprint(breed_cat_dict, indent=4)

##### Owner and Dog Characteristics

Next, we focus on transforming and creating features related to owner and dog characteristics. This involves extracting information such as gender, age group, and district details.


In [ ]:
zurich_dog_data["mixed_type"] = zurich_dog_data["breed_mongrel_long"].map(
    breed_cat_dict
)
zurich_dog_data["owner_id"] = zurich_dog_data["holder_id"].astype(
    str).str.zfill(6)
# rename the mixed types to something more manageable
zurich_dog_data["mixed_type"] = zurich_dog_data["mixed_type"].map(
    {
        "Pedigree dog": "PB",
        "Mixed breed, both breeds known": "BB",
        "Mixed breed, secondary breed unknown": "BU",
        "Mixed breed, both breeds unknown": "UU",
    }
)

In [ ]:
# Define owner and dog gender
zurich_dog_data["is_male_owner"] = zurich_dog_data["sex_cd"] == 1
zurich_dog_data["is_male_dog"] = zurich_dog_data["sex_dog_cd"] == 1

# Copy and modify age group
zurich_dog_data["age_group_10"] = zurich_dog_data["age_v_10_cd"].copy()


# Copy necessary columns
zurich_dog_data["dog_age"] = zurich_dog_data["age_v_dog_cd"].copy().astype(int)
zurich_dog_data["district"] = zurich_dog_data["circle_cd"].copy().astype(int)
zurich_dog_data["sub_district"] = zurich_dog_data["quar_cd"].copy().astype(
    "string")
zurich_dog_data["breed_type"] = zurich_dog_data["breed_type_cd"].copy()
zurich_dog_data["roster"] = zurich_dog_data["deadline_date_year"].copy()

# Calculate total dogs per owner and roster
zurich_dog_data["pet_count"] = zurich_dog_data.groupby(["owner_id", "roster"])[
    "number_of_dogs"
].transform(np.sum)

# Define breed status
zurich_dog_data["is_pure_breed"] = zurich_dog_data["breed_mixed__breed_sort"] == 1
# zurich_dog_data["is_mixed_breed"] = ~zurich_dog_data["is_pure_breed"]

# Copy breed and neighborhood information
zurich_dog_data["breed_1"] = zurich_dog_data["race_1_text"].copy()
zurich_dog_data["breed_2"] = zurich_dog_data["race_2_text"].copy()
zurich_dog_data["neighborhood"] = zurich_dog_data["quar_lang"].copy().astype(
    "string")

# Copy dog color
zurich_dog_data["dog_color"] = zurich_dog_data["dog_color_text"].copy()

# Create new DataFrame with selected columns
columns_of_interest = [
    "owner_id",
    "is_male_owner",
    "is_male_dog",
    "age_group_10",
    "dog_age",
    "district",
    "sub_district",
    "neighborhood",
    "roster",
    "pet_count",
    "number_of_dogs",
    "mixed_type",
    "is_pure_breed",
    "breed_type",
    "breed_1",
    "breed_2",
    "dog_color",
]
dog_data = zurich_dog_data[columns_of_interest].copy()

In [ ]:
dog_data.sample(3)
# dog_data.age_group_10.value_counts()
dog_data.isna().sum()

##### Filtering Doodle Dogs

A specific analysis is conducted to filter out dogs with 'doodle' in their breed names, converting them to mixed breeds and updating breed information accordingly. This is a designer breed which is not yet recognized.


In [ ]:
# Create mask to filter out the doodle dogs
doodle_mask = dog_data["breed_1"].str.contains(
    r".*doodle", regex=True, na=False, case=False
)
print(f"Number of doodle dogs: {doodle_mask.sum()}")
# convert them to mixed breed if they are pure breeds
dog_data.loc[doodle_mask, "is_pure_breed"] = False
dog_data.loc[doodle_mask, "breed_2"] = "Pudel"
dog_data.loc[doodle_mask, "mixed_type"] = "BB"
dog_data.loc[doodle_mask, "breed_1"] = dog_data.loc[doodle_mask, "breed_1"].apply(
    lambda x: "Golden Retriever" if x.startswith("G") else "Labrador Retriever"
)
# dog_data[doodle_mask]

##### Translations

To enhance clarity and conciseness, the translation process for columns such as `dog_color`, `breed_1`, and `breed_2` has been streamlined. The following code efficiently performs the translations:

In [ ]:
# Unique values for dog colors
dog_colors = dog_data["dog_color"].str.lower().unique()

# Translate dog colors
dog_color_translations = translate_list_to_dict(dog_colors)
dog_data["dog_color_en"] = dog_data["dog_color"].str.lower().map(
    dog_color_translations)

# Unique values for breed_1
breeds_1 = dog_data["breed_1"].str.lower().unique()

# Translate breed_1
breed_1_translations = translate_list_to_dict(breeds_1)
dog_data["breed_1_en"] = dog_data["breed_1"].str.lower().map(
    breed_1_translations)

# Unique values for breed_2
breeds_2 = dog_data["breed_2"].str.lower().unique()

# Translate breed_2
breed_2_translations = translate_list_to_dict(breeds_2)
dog_data["breed_2_en"] = dog_data["breed_2"].str.lower().map(
    breed_2_translations)

In [ ]:
dog_data.sample(3)

##### Breed Standardization
To ensure consistency in the analysis, the breeds in the dataset are standardized. Since the "breed" column is free text, allowing dog owners to input their breed information during registration, variations can exist even for the same breeds. To address this, a data frame is employed, containing breeds recognized by the FCI (Fédération Cynologique Internationale). Within this data frame, each recognized FCI breed has a column listing its name in different languages and alternative, unofficial names. This approach helps capture variations in breed names and facilitates grouping similar breeds together.

In [ ]:
# Get the FCI dataframe with the recognized breeds
fci_breeds = pd.read_json("../data/fci_breeds.json")
fci_breeds[["alt_names", "breed_en"]]

# Create a DataFrame with translated breed names
breeds_df = pd.DataFrame.from_dict(
    {**breed_1_translations, **breed_2_translations}, orient="index"
).reset_index()
breeds_df.columns = ["breed_de", "breed_en"]

# Initialize a "standard" column for breed standardization
breeds_df["standard"] = None
nan_mask = breeds_df["standard"].isna()

# Match each column for breed standardization
for col in breeds_df.columns:
    matched_value = hf.apply_fuzzy_matching_to_breed_column(
        breeds_df.loc[nan_mask], col, fci_breeds, [fuzz.WRatio]
    )
    breeds_df.loc[nan_mask, "standard"] = matched_value[nan_mask]
    nan_mask = breeds_df["standard"].isna()

This process involves leveraging fuzzy matching techniques to match the translated breed names with the FCI breeds, providing a standardized representation of each breed.


Following the application of fuzzy logic for breed standardization, most breeds were successfully matched. However, a few rare breeds, those categorized as unknown or mixed, and a specific variation of the Welsh Corgi breed remain unmatched, as they are not recognized by the FCI. Additionally, there is a breed labeled as "eel," which is also not officially recognized. To address this, the values from the `breed_en` column are copied to the `standard` column for the unmatched breeds. Furthermore, the "elo" breed is explicitly set in the standard column due to potential mismatches.



This ensures that even the unmatched and non-FCI recognized breeds are accounted for in the standardized breed names.

In [ ]:
# Update the standard column for specific cases
breeds_df.loc[nan_mask, "standard"] = breeds_df.loc[nan_mask, "breed_en"]
breeds_df.loc[breeds_df["breed_de"] == "elo", "standard"] = "elo"
breeds_df.loc[breeds_df["breed_de"] == "keine", "standard"] = "none"
breeds_df.loc[breeds_df["breed_de"] == "mischling", "standard"] = "hybrid"

# Convert breed_1 to lowercase for merging
dog_data["breed_1"] = dog_data["breed_1"].str.lower()
dog_data["breed_2"] = dog_data["breed_2"].str.lower()

# Merge with the breeds_df for standardized breed names
dog_data = dog_data.merge(
    breeds_df.drop(columns=["breed_en"]),
    left_on="breed_1",
    right_on="breed_de",
    suffixes=("", "_1"),
)

dog_data = dog_data.merge(
    breeds_df.drop(columns=["breed_en"]),
    left_on="breed_2",
    right_on="breed_de",
    suffixes=("", "_2"),  # Add suffix to distinguish columns
)

Final missing values for the `age_group_10` column which represents the the age groups of the dog owners. for these we fill in the value of -1 but also make a note of the rows that have missing values for this column. We also fill in the age group when possible if the owner had shared that information in later year's rosters. We then create a new column called `age_group_20` which is the age group of the dog owners in 20-year increments. These larger age groups would help to simplify our model while still giving valuable information about the age of the dog owners.

In [ ]:
# Create an indicator variable for missing 'age_group_10' values
dog_data["age_group_missing"] = dog_data["age_group_10"].isna().astype(int)

# Fill in the missing 'age_group_10' values
dog_data["age_group_10"] = dog_data["age_group_10"].fillna(
    dog_data.groupby("owner_id")["age_group_10"].transform(
        lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan
    )
)

dog_data["age_group_10"] = dog_data["age_group_10"].fillna(-1).astype(int)
dog_data["age_group_20"] = dog_data["age_group_10"].apply(
    lambda x: -1 if x == -1 else math.floor(x / 20) * 20
)

##### Expanding the dogs Dataset
There are rows in the dogs dataset which represent more than one dog. The number of dogs for that row is given in the `number_of_dogs` column. We expand the dataset to have one dog for each row. This is done by repeating the rows by the number in the `number_of_dogs` column. We reset the index after so that we have a unique index for each row.

In [ ]:
# Repeat each row based on the number of dogs in the row
expanded_dog_data = dog_data.loc[
    np.repeat(dog_data.index.values, dog_data["number_of_dogs"])
]
expanded_dog_data.drop(columns=["number_of_dogs"], inplace=True)
expanded_dog_data.reset_index(drop=True, inplace=True)

#### Consolidated Data

The resulting dataset, `dog_data`, is a refined version containing selected columns of interest. This dataset will be used for further analysis and modeling.


In [ ]:
expanded_dog_data.dtypes

In [ ]:
dog_data_columns_to_keep = [
    "owner_id",
    "is_male_owner",
    "is_male_dog",
    "age_group_10",
    "age_group_20",
    "dog_age",
    "district",
    "sub_district",
    "neighborhood",
    "roster",
    "pet_count",
    "mixed_type",
    "is_pure_breed",
    "breed_type",
    "dog_color_en",
    "breed_2_en",
    "standard",
    "age_group_missing",
]

filtered_dog_data = expanded_dog_data.query(
    "roster < 2023")[dog_data_columns_to_keep]
filtered_dog_data.to_csv("../data/processed_dog_data.csv", index=False)

filtered_dog_data.sample(10)

Now we look at some of the descriptive statistics for the `dog_data` dataset just to understand the distribution of some of the characteristics.
Each year here is a increase in the total number of dogs kept as pets With the majority of pet owners having only one dog.

In [ ]:
processed_dog_data_path = "../data/processed_dog_data.csv"
filtered_dog_data = pd.read_csv(processed_dog_data_path)

In [ ]:
filtered_dog_data.describe().T

In [ ]:
filtered_dog_data.describe(include="all").T.sort_values(by="unique").fillna("")
filtered_dog_data.info()
filtered_dog_data.query('neighborhood.str.contains("Albisrieden")')
filtered_dog_data["sub_district"] = (
    filtered_dog_data["sub_district"].astype(int).astype("string").str.zfill(3)
)

In [ ]:
def update_xaxis(plot, element):
    """Hook to update the x-axis ticker on the plot."""
    plot.state.xaxis.ticker = FixedTicker(ticks=list(range(2015, 2023)))


dogs_total_by_roster = filtered_dog_data.groupby("roster").size()
print(f"Total number of dogs per year:\n{dogs_total_by_roster}")

total_dogs_line = dogs_total_by_roster.hvplot.bar().opts(
    show_legend=False,
    title="Total Dogs Registered Each Year",
    active_tools=["box_zoom"],
    # height=500,
    # width=400,
)

dog_count_yoy_pct_change = dogs_total_by_roster.pct_change().fillna(0) * 100
total_dogs_yoy_bar = dog_count_yoy_pct_change.hvplot(kind="line").opts(
    hooks=[update_xaxis],
    active_tools=["box_zoom"],
    title="YOY % Change in Dog Count",
    ylabel="%",
)

(total_dogs_line + total_dogs_yoy_bar).cols(1).opts(shared_axes=False)

We can also look at these distributions spatially using a chloropleth map for the number of dogs and for the dog density, dogs per unit area. This answers the question what is the spatial distribution of dogs in theh different neighborhoods across the city of Zurich for each of the years.

In [ ]:
# Create a player widget
# yearly_player = pnw.Player(
#     name="Yearly Player",
#     start=2015,
#     end=2022,
#     value=2020,
#     step=1,
#     loop_policy="loop",
#     interval=3000,
# )
# Create a slider for the roster
roster_slider = pnw.IntSlider(value=2020, start=2015, end=2022)

In [ ]:
@pn.depends(roster_slider.param.value)
def get_neighborhood_dog_density(roster):
    # Get the total dog count in each neighborhood for each roster
    roster_dog_data = filtered_dog_data.query(f"roster=={roster}")
    neighborhood_total_dog_count = (
        roster_dog_data.groupby(["neighborhood"]).size().reset_index(name="total_dogs")
    ).set_index("neighborhood")

    map_gdf = merged_gdf.set_index("neighborhood")

    roster_dog_data_gdf = map_gdf.merge(
        neighborhood_total_dog_count, left_index=True, right_index=True, how="left"
    )
    roster_dog_data_gdf["dog_density"] = (
        roster_dog_data_gdf["total_dogs"] / roster_dog_data_gdf["area_km2"]
    )

    return gv.Polygons(roster_dog_data_gdf).opts(
        **poly_opts,
        color="dog_density",
        colorbar=True,
        tools=["hover", "tap", "box_select"],
        color_levels=6,
        title=f"Dog Density {roster} [dogs/km2]",
    )


@pn.depends(roster_slider.param.value)
def get_neighborhood_dogs_total(roster):
    df = filtered_dog_data.query(f"roster=={roster}")
    df = df.groupby("neighborhood").size().reset_index(name="total_dogs")
    df = df.set_index("neighborhood")

    map_gdf = neighborhood_gdf.set_index("neighborhood")

    dog_gdf = map_gdf.merge(df, left_index=True, right_index=True, how="left")
    return gv.Polygons(dog_gdf).opts(
        **poly_opts,
        color="total_dogs",
        colorbar=True,
        tools=["hover", "tap", "box_select"],
        color_levels=6,
        title=f"Dogs Distribution || {roster}",
    )


dog_density_pane = pn.pane.HoloViews(get_neighborhood_dog_density)
neighborhood_dog_count_pane = pn.pane.HoloViews(get_neighborhood_dogs_total)
# Shows a Row with the count and the density side by side
# pn.Column(roster_slider, pn.Row(neighborhood_dog_count_pane, dog_density_pane))
# dog_density_pane
# neighborhood_dog_count_pane

The Butterfly plot here shows the The edge distribution of male and female dogs for each of the years. We can see here that both genders tend to live around the same length of time we also see that there are slightly more male dogs than female dogs.

In [ ]:
dog_age_distribution_pane = pn.pane.HoloViews(hf.get_dog_age_butterfly_plot)
pn.Column(
    hf.yearly_player,
    dog_age_distribution_pane,
    sizing_mode="stretch_width",
)

In [ ]:
hf.get_line_plots(
    filtered_dog_data.groupby("roster", as_index=False)["mixed_type"].value_counts(),
    x="roster",
    group_by="mixed_type",
    highlight_list=["PB"],
).opts(hooks=[update_xaxis], title="Most Dogs are Pure Breeds", xlabel="")

In [ ]:
hf.get_line_plots(
    filtered_dog_data.groupby("roster", as_index=False)[
        "breed_type"].value_counts(),
    x="roster",
    group_by="breed_type",
    highlight_list=["K"],
).opts(hooks=[update_xaxis], title="More Dogs are Small Breeds", xlabel="")

Here we see that there is a strong upward tick in the number of dog owners for 30 and 40 year-old. Up until 2019 it was the 50 year olds that had the highest number of dog owners This is a trend that we will explore further in this analysis.

In [ ]:
highlighted_age_groups = [20, 40]
dog_owner_grouped_count = filtered_dog_data.groupby(["roster", "age_group_20"])[
    "owner_id"
].nunique()
dog_owner_grouped_count

plot = hf.get_line_plots(
    dog_owner_grouped_count.reset_index(),
    "roster",
    "age_group_20",
    highlighted_age_groups,
)
plot.opts(
    title="Young Adults & Middle-Age Dog Owners count is trending upwards",
    height=500,
    width=800,
    hooks=[update_xaxis],
    xlabel="",
    ylabel="",
)

Popular Breeds


In [ ]:
# the top n small breeds in Zurich
ktopn = (
    filtered_dog_data.loc[filtered_dog_data["breed_type"] == "K"]["standard"]
    .value_counts()
    .head(15)
    .index.tolist()
)
ktopn_pure = (
    filtered_dog_data.loc[
        (filtered_dog_data["breed_type"] == "K") & (
            filtered_dog_data["is_pure_breed"])
    ]
    .standard.value_counts()
    .head(15)
    .index.tolist()
)

# The top n big breeds in Zurich
itopn = (
    filtered_dog_data[filtered_dog_data["breed_type"] == "I"]["standard"]
    .value_counts()
    .head(15)
    .index.tolist()
)
itopn_pure = (
    filtered_dog_data.loc[
        (filtered_dog_data["breed_type"] == "I") & (
            filtered_dog_data["is_pure_breed"])
    ]
    .standard.value_counts()
    .head(15)
    .index.tolist()
)

topn = ktopn + itopn
topn_pure = ktopn_pure + itopn_pure

In [ ]:
mask_path = "../images/dalle_dog_wordcloud.png"
dog_image = hv.RGB.load_image(mask_path)
threshold = 0.7
dog_image_mask = dog_image.data
dog_image_mask[dog_image_mask > threshold] = 1

# Create a count of the various pure breeds
pure_breed_count = filtered_dog_data.loc[
    (filtered_dog_data["is_pure_breed"])
].standard.value_counts()

# Create a word cloud for the pure breeds
wc = WordCloud(
    max_font_size=66,
    contour_width=10,
    contour_color="steelblue",
    background_color="white",
    colormap="bone",
    mask=(dog_image_mask * 255).astype(np.uint8),
).generate_from_frequencies(pure_breed_count)


breed_wordcoud = hv.RGB(wc.to_array()).opts(
    width=wc.width,
    height=wc.height,
    title="Breed Popularity",
    show_frame=False,
    xaxis=None,
    yaxis=None,
    # bgcolor="gray",
    padding=0.05,
    active_tools=["box_zoom"],
)
pn.pane.HoloViews(breed_wordcoud)

#### Zurich Population Dataset



In [ ]:
# zurich_pop_link = "https://data.stadt-zuerich.ch/dataset/bev_bestand_jahr_quartier_alter_herkunft_geschlecht_od3903/download/BEV390OD3903.csv"
zurich_pop_link = "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_pop.csv"
zurich_pop_data = pd.read_csv(zurich_pop_link)
zurich_pop_data = hf.sanitize_df_column_names(zurich_pop_data)
zurich_pop_data.info()

In [ ]:
zurich_pop_data.sample().T

In [ ]:
# Create 'is_swiss' column, True if 'origin_lang' contains 'Schweizer'
zurich_pop_data["is_swiss"] = zurich_pop_data["origin_lang"].str.contains(
    "Schweizer", regex=False, na=False, case=False
)

# Create new columns with copied data for further processing
zurich_pop_data["neighborhood"] = zurich_pop_data["quar_lang"].copy()
zurich_pop_data["district"] = zurich_pop_data["circle_cd"].copy().astype(int)

# Create 'sub_district' column, ensuring it's a string with leading zeros
zurich_pop_data["sub_district"] = (
    zurich_pop_data["quar_cd"].copy().astype(str).str.zfill(3)
)

# Create new columns with copied data for further processing
zurich_pop_data["roster"] = zurich_pop_data["deadline_date_year"].copy()
zurich_pop_data["age_group_10"] = zurich_pop_data["age_v_10_cd"].copy()
zurich_pop_data["age_group_20"] = zurich_pop_data["age_v_20_cd"].copy()
zurich_pop_data["age"] = zurich_pop_data["age_v_cd"].copy()

# Create 'is_male' column, True if 'sex_cd' equals 1
zurich_pop_data["is_male"] = zurich_pop_data["sex_cd"] == 1

# Create 'pop_count' column with copied data from 'number_we'
zurich_pop_data["pop_count"] = zurich_pop_data["number_we"].copy()

# Define a list of columns of interest for the final dataframe
columns_of_interest_pop = [
    "is_swiss",
    "neighborhood",
    "district",
    "sub_district",
    "roster",
    "age_group_10",
    "age_group_20",
    "age",
    "is_male",
    "pop_count",
]

# Create a new dataframe 'pop_data' with only the columns of interest
pop_data = zurich_pop_data[columns_of_interest_pop].copy()

# Display the structure of 'pop_data' dataframe for verification
pop_data.info()
pop_data.to_csv("../data/processed_pop_data.csv", index=False)

We need to ensure That there were datasets have some of the same column and column names so that they could be merged together and matched well. Since we only have data from the year 2015 with adults data set we will only look at data from the year 2015 with the population data set. And to match the age range that we have in the dogs data set we will also take the ages in the 10 year range here in addition to that the ado is also given on at the neighborhood level so that can be easily matched as well.


In [ ]:
pop_data = pd.read_csv("../data/processed_pop_data.csv")

In [ ]:
pop_data.sample(10)
pop_data.query('neighborhood.str.contains("Albisrieden") & roster==2015')
pop_data.info()
pop_data.sample().T
pop_data["sub_district"] = pop_data["sub_district"].astype(str).str.zfill(3)

Let's look at some descriptive statistics on the data first.Here we see that the data is pretty clean already there are only 30 four neighborhoods and sub districts there are 12 districts come on the Rasta who is from 1993 to 2022 however we would only use from 2015. We will aggregate the data for the ages so that it matches the range that we had in the dog data set so that our direct comparison can be made.

In [ ]:
# Description of the data
pop_data.describe(include="all").T.sort_values(by="unique").fillna("")

In [ ]:
pop_data.loc[pop_data["roster"] >= 2015]
post_2015_pop_data = pop_data.loc[pop_data["roster"] >= 2015].copy()

# group by age group , roster, subdistrict, is_male and sum the population count
pop_gender_agg = (
    post_2015_pop_data.groupby(
        ["roster", "sub_district", "is_male", "age_group_20", "neighborhood"]
    )["pop_count"]
    .sum()
    .reset_index()
)
# quick sanity check to see if the sum is what we believe the population of Zurich is
pop_gender_agg.groupby("roster")["pop_count"].sum()

In [ ]:
# Get the unique age groups from the dog data
dog_age_groups = filtered_dog_data.age_group_20.unique()

# Filter the population data to only include these age groups
pop_data_filtered = pop_gender_agg[pop_gender_agg.age_group_20.isin(
    dog_age_groups)]
pop_data_filtered.head(5)

In [ ]:
# Do a aggregation of the pop data on the roster level and on the is_male level and sum the population count
pop_gender_agg = (
    pop_data_filtered.groupby(["roster", "is_male"])[
        "pop_count"].sum().unstack()
)

# Normalize the population count to calculate the percentage portion
pop_data_agg_normalized = pop_gender_agg.div(
    pop_gender_agg.sum(axis=1), axis=0)

pop_data_agg_normalized.round(3)

We see that they're just about equal amounts of men and women male and female persons in Zurich this is interesting because the dog owners ratio but significantly higher for females than for males.

In [ ]:
# Do a sum per roster and per neighborhood and display graphically
pop_by_neighborhood = (
    pop_data_filtered.groupby(["roster", "sub_district"])[
        "pop_count"].sum().unstack()
)
pop_data_sub_district_normalized = pop_by_neighborhood.div(
    pop_by_neighborhood.sum(axis=1), axis=0
)
plt.figure(figsize=(10, 10))
# Create the heatmap
pop_heatmap = sns.heatmap(
    pop_data_sub_district_normalized.T,
    cmap="Blues",
    cbar=True,
    linewidths=0.5,
    linecolor="white",
).set(xlabel="", ylabel="")

# Rotate the x-axis labels
plt.xticks(rotation=0)
# Rotate the y-axis labels
plt.yticks(rotation=0)
plt.title("Normalized Neighborhood Population")

plt.show()

The heat map shows that the proportion of the population within each of the neighborhoods in Zurich stayed approximately the same throughout the years from 2015 to 2022.

In [ ]:
# pop_data_filtered
@pn.depends(roster_slider.param.value)
def get_pop_neighborhood_count(roster):
    df = pop_data_filtered.query(f"roster=={roster}")
    df = df.groupby("neighborhood")["pop_count"].sum()
    map_gdf = neighborhood_gdf.set_index("neighborhood")
    agg_gdf = map_gdf.merge(df, left_index=True, right_index=True, how="left")
    return gv.Polygons(agg_gdf).opts(
        **poly_opts,
        color="pop_count",
        colorbar=True,
        tools=["hover", "tap", "box_select"],
        title=f"Pop Count {roster}",
        aspect="equal",
        color_levels=6,
    )


@pn.depends(roster_slider.param.value)
def get_pop_density(roster):
    df = pop_data_filtered.query(f"roster=={roster}")
    df = df.groupby("neighborhood")["pop_count"].sum()
    map_gdf = merged_gdf.set_index("neighborhood")
    agg_gdf = map_gdf.merge(df, left_index=True, right_index=True, how="left")
    agg_gdf["pop_density"] = agg_gdf["pop_count"] / agg_gdf["area_km2"]
    return gv.Polygons(agg_gdf).opts(
        **poly_opts,
        color="pop_density",
        colorbar=True,
        tools=["hover", "tap", "box_select"],
        title=f"Pop Density {roster}",
        aspect="equal",
        color_levels=6,
    )


pop_count_pane = pn.pane.HoloViews(get_pop_neighborhood_count)
pop_density_pane = pn.pane.HoloViews(get_pop_density)

# pn.Column(roster_slider, pn.Row(pop_count_pane, pop_density_pane))

In [ ]:
# Place the Dog density and population density plots on same page for comparison
pn.Column(
    roster_slider,
    pn.Row(neighborhood_dog_count_pane, dog_density_pane),
    pn.Row(pop_count_pane, pop_density_pane),
)

#### Comparing the Population and Dog Owner population

With the population data aggregated for every 10 year interval similarly to the dog owners data set we can compare these two datasets to see if any correlation exists between the two (count or density). We can also measure to see if there's any correlation or similarities in the proportions based on the age or the gender of the dog owners and the population or how this has changed year over year

In [ ]:
(
    filtered_dog_data.groupby(
        ["roster", "is_male_owner"],
    )["owner_id"]
    .nunique()
    .unstack()
)
filtered_dog_data

In [ ]:
# Compare the gender of the dog Owners for each year
dog_owner_gender_agg = (
    filtered_dog_data.groupby(
        ["roster", "is_male_owner"],
    )["owner_id"]
    .nunique()
    .unstack()
)
# normalize the count by the total dog_owner count per roster
dog_owner_gender_agg_normalized = dog_owner_gender_agg.div(
    dog_owner_gender_agg.sum(axis=1), axis=0
)
# dog_owner_agg_normalized

# Do a aggregated count by subdistrict/neighborhood and roster
filtered_dog_data["sub_district"] = (
    filtered_dog_data["sub_district"].astype(int).astype("string").str.zfill(3)
)
dog_owner_subdistrict = (
    filtered_dog_data.groupby(
        [
            "roster",
            "sub_district",
        ]
    )["owner_id"]
    .nunique()
    .unstack()
)
# Normalized the count by the total count per roster
dog_owner_subdistrict_normalized = dog_owner_subdistrict.div(
    dog_owner_subdistrict.sum(axis=1), axis=0
)
dog_owner_subdistrict_normalized

In [ ]:
# print(f"Dog Owner Count by Sub District")
# display(dog_owner_subdistrict)
# print(f"Population Count by Sub District")
# display(pop_by_neighborhood)

In [ ]:
# Check the percentage change from the previous year
dog_owner_subdistrict.pct_change(axis=0).fillna(0)

# merge the dog owner count and the population count on the roster and subdistrict
dog_pop_comb_dataset = (
    # dog_data.query("roster < 2023")
    filtered_dog_data.groupby(["roster", "neighborhood", "sub_district"])[
        ["owner_id"]]
    .nunique()
    .merge(
        pop_data_filtered.groupby(["roster", "neighborhood", "sub_district"])[
            ["pop_count"]
        ].sum(),
        left_index=True,
        right_index=True,
    )
)
dog_pop_comb_dataset = dog_pop_comb_dataset.rename(
    columns={"owner_id": "owner_count"})

dog_pop_comb_dataset["dog_owner_pop_ratio"] = (
    dog_pop_comb_dataset["owner_count"] / dog_pop_comb_dataset["pop_count"]
)

dog_pop_comb_dataset["dog_owner_pop_ratio_pct_change"] = (
    dog_pop_comb_dataset.groupby(["neighborhood", "sub_district"])[
        "dog_owner_pop_ratio"
    ]
    .pct_change(axis=0)
    .fillna(0)
)
dog_pop_comb_dataset.sample(5)

In [ ]:
dog_pop_comb_dataset["dog_owner_pop_ratio"].reset_index()

In [ ]:
dog_pop_comb_dataset.describe().T

hv.HeatMap(
    dog_pop_comb_dataset["dog_owner_pop_ratio"]
    .reset_index()
    .sort_values(by="sub_district", ascending=False),
    kdims=[
        "roster",
        "sub_district",
    ],
    vdims=["dog_owner_pop_ratio", "neighborhood"],
).opts(
    height=800,
    width=500,
    title="Dog Owner to Population Ratio",
    tools=["hover"],
    colorbar=True,
    color_levels=4,
    hooks=[update_xaxis],
    active_tools=["box_zoom"],
)

In [ ]:
highlighted_sub_districts = ["074", "071", "052", "072"]
hf.get_line_plots(
    dog_pop_comb_dataset["dog_owner_pop_ratio"].reset_index(),
    x="roster",
    group_by="sub_district",
    highlight_list=highlighted_sub_districts,
).opts(
    height=800,
    width=600,
    legend_cols=2,
    title="Dog Owner to Population Ratio by Sub District",
    hooks=[update_xaxis],
)

In [ ]:
# Create a figure with two subplots and adjust the figure size
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Create the first heatmap for dog ownership
sns.heatmap(
    dog_owner_subdistrict_normalized.T,
    cmap="Blues",
    cbar=False,
    linewidths=2,
    linecolor="white",
    ax=ax1,  # Specify the first subplot as the target
)
ax1.set_title("Dog Ownership Trend in Zurich")
ax1.set(xlabel="", ylabel="")
ax1.title.set_size(16)
ax1.tick_params(axis="both", which="major", labelsize=12)

# Create the second heatmap for population dynamics
sns.heatmap(
    pop_data_sub_district_normalized.T,
    cmap="Blues",
    cbar=True,  # Enable the color bar for the second heatmap
    linewidths=2,
    linecolor="white",
    ax=ax2,  # Specify the second subplot as the target
)
ax2.set_title("Zurich Population Dynamics")
ax2.set(xlabel="", ylabel="")
ax2.title.set_size(16)
ax2.tick_params(axis="both", which="major", labelsize=12)

# Improve the layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
custom_palette = sns.color_palette(["pink", "skyblue"])


# Create a figure with two subplots and adjust the figure size
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 7))

# Plot the first bar plot for dog owners
dog_owner_gender_agg_normalized.plot.barh(
    stacked=True,
    title="More Female than Male Dog Owners",
    ax=ax1,
    color=custom_palette,
    legend=False,
)
ax1.set(xlabel="", ylabel="")
ax1.title.set_size(16)  # Increase title font size
# Increase tick label size
ax1.tick_params(axis="both", which="major", labelsize=12)

# Plot the second bar plot for population
pop_data_agg_normalized.plot.barh(
    stacked=True,
    title="Even Female & Male Population",
    ax=ax2,
    color=custom_palette,
    legend=False,
)
ax2.set(xlabel="", ylabel="")
ax2.title.set_size(16)
ax2.tick_params(axis="both", which="major", labelsize=12)

# Improve the layout
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
pop_data_filtered.info()

In [ ]:
# Group the population count by roster, sub_district, is_male, and age_group_10
pop_count_grouped_4_cols = (
    pop_data_filtered.groupby(["roster", "sub_district", "is_male", "age_group_20"])[
        "pop_count"
    ]
    .sum()
    .reset_index()
)

# Display the grouped population count
pop_count_grouped_4_cols.head()

In [ ]:
filtered_dog_data

In [ ]:
# Group by roster, sub_district, is_male_owner, and age_group_10
grouped_4_cols = filtered_dog_data.groupby(
    ["roster", "sub_district", "is_male_owner", "age_group_20"]
)

# Count the number of dogs and unique owners in each group
dog_count_grouped_4_cols = (
    grouped_4_cols.size()
    .reset_index(name="dog_count")
    .rename(columns={"is_male_owner": "is_male"})
)

dog_owner_grouped_4_cols = (
    grouped_4_cols["owner_id"]
    .nunique()
    .reset_index(name="owner_count")
    .rename(columns={"is_male_owner": "is_male"})
)

##### Merging Population and Dog Owners Data

We grouped `filtered_dog_data` by 'roster', 'sub_district', 'is_male_owner', and 'age_group_10' to form unique groups. 

Two datasets were created:

1. `dog_count_grouped_4_cols`: Contains dog counts per group, calculated using `size`.
2. `dog_owner_grouped_4_cols`: Contains unique owner counts per group, calculated using `nunique` on 'owner_id'.

The 'is_male_owner' column was renamed to 'is_male' for consistency.

These datasets summarize dog and owner counts for further analysis.

In [ ]:
# Merge the grouped population count and dog owner count dataframes
merged_grouped_4_cols = pop_count_grouped_4_cols.merge(
    dog_owner_grouped_4_cols,
)
merged_grouped_4_cols["owner_count"] = merged_grouped_4_cols["owner_count"].fillna(
    0)

# Calculate the ratio of dog owners to population for each group
merged_grouped_4_cols["owner_pop_ratio"] = (
    merged_grouped_4_cols["owner_count"] / merged_grouped_4_cols["pop_count"]
)
# Drop the Nans rows
merged_grouped_4_cols.dropna(inplace=True)
merged_grouped_4_cols.head(10)

In [ ]:
described_data = merged_grouped_4_cols.groupby(["age_group_20", "sub_district"])[
    ["owner_pop_ratio"]
].describe()

described_data.columns = ["_".join(col)
                          for col in described_data.columns.values]

# Reset the index to make 'age_group_20' and 'sub_district' regular columns
described_data.reset_index(inplace=True)
columns_for_plot = [
    "age_group_20",
    "sub_district",
    "owner_pop_ratio_mean",
    "owner_pop_ratio_50%",
    "owner_pop_ratio_std",
]
described_data = described_data[columns_for_plot].sort_values(
    by=["sub_district"], ascending=False
)
age_group_sub_district_heatmap = hv.HeatMap(
    described_data,
    kdims=["age_group_20", "sub_district"],
    vdims=["owner_pop_ratio_mean", "owner_pop_ratio_50%", "owner_pop_ratio_std"],
).opts(
    height=800,
    width=500,
    line_color="white",
    color_levels=6,
    tools=["hover", "tap"],
    colorbar=True,
    active_tools=["box_zoom"],
    nonselection_fill_alpha=0.5,
    title="Mean Dog Owner to Population Ratio by Age Group and Sub District",
)
described_data
# age_group_sub_district_heatmap

In [ ]:
merged_grouped_4_cols

In [ ]:
# Create a Tap stream on the heatmap
tap_stream = streams.Tap(source=age_group_sub_district_heatmap, y="012", x=40)


@pn.depends(tap_stream.param.x, tap_stream.param.y)
def get_bar_plot_by_age_group_and_sub_district(x, y):
    # Return a message if no location is tapped
    if x is None or y is None:
        return pn.pane.Markdown("No sub district selected")
    else:
        # Convert tapped location to sub_district and age_group
        sub_district = str(y)
        age_group = math.floor(x / 20) * 20
        # Filter dataframe based on sub_district and age_group
        filtered_df = merged_grouped_4_cols.loc[
            (merged_grouped_4_cols["sub_district"] == sub_district)
            & (merged_grouped_4_cols["age_group_20"] == age_group)
        ]
        # Convert is_male column to string
        filtered_df["is_male"] = filtered_df["is_male"].apply(
            lambda x: "Male" if x else "Female"
        )
        # Create bar plot of filtered dataframe
        bar_plot = filtered_df.hvplot.bar(
            x="roster",
            y="owner_pop_ratio",
            title=f"Dog Owner to Population Ratio for {age_group} in {sub_district}",
            xlabel="Year",
            ylabel="Dog Owner to Population Ratio",
            by="is_male",
            width=500,
            height=800,
            stacked=True,
            color=["pink", "skyblue"],
        ).opts(
            active_tools=["box_zoom"],
        )
        # Return bar plot
        return bar_plot


# Create Panel object to display bar plot
bar_panel = pn.pane.HoloViews(get_bar_plot_by_age_group_and_sub_district)
# Display heatmap and bar plot side by side
pn.Row(age_group_sub_district_heatmap, bar_panel)

In [ ]:
# tap_stream = streams.Tap(source=age_group_sub_district_heatmap, y="012", x=50)


# @pn.depends(tap_stream.param.x, tap_stream.param.y)
# def get_bar_plot_by_age_group_and_sub_district(x, y):
#     if x is None or y is None:
#         return pn.pane.Markdown("No sub district selected")
#     else:
#         sub_district = str(y)
#         age_group = math.floor(x / 10) * 10
#         filtered_df = merged_grouped_4_cols.loc[
#             (merged_grouped_4_cols["sub_district"] == sub_district)
#             & (merged_grouped_4_cols["age_group_10"] == age_group)
#         ]
#         filtered_df['is_male'] = filtered_df['is_male'].apply(
#             lambda x: 'Male' if x else 'Female')
#         bar_plot = filtered_df.hvplot.bar(
#             x="roster",
#             y="owner_pop_ratio",
#             title=f"Dog Owner to Population Ratio for {age_group} in {sub_district}",
#             xlabel="Year",
#             ylabel="Dog Owner to Population Ratio",
#             by="is_male",
#             # ylim=(0, 0.3),
#             # rot=90,
#             width=500,
#             height=800,
#             stacked=True,
#             color=['pink', "skyblue"],
#         ).opts(
#             active_tools=["box_zoom"],
#         )
#         return bar_plot


# bar_panel = pn.pane.HoloViews(get_bar_plot_by_age_group_and_sub_district)
# pn.Row(age_group_sub_district_heatmap, bar_panel)

In [ ]:
described_data.loc[
    described_data["sub_district"].isin(highlighted_sub_districts)
].hvplot(
    x="age_group_20",
    y="owner_pop_ratio_mean",
    by="sub_district",
    kind="bar",
    height=600,
    width=1200,
    line_color="white",
    tools=["hover"],
    colorbar=True,
    title=f"Mean Dog Owner to Population Ratio by Age Group and Sub District",
).opts(
    active_tools=["box_zoom"]
)

In [ ]:
# Create boxplot to show distribution of 'owner_pop_ratio' across 'age_group_10'
sns.boxplot(x="age_group_20", y="owner_pop_ratio", data=merged_grouped_4_cols)

plt.title(f"Owner to Population Ratio by Age Group")
plt.show()

In [ ]:
merged_grouped_4_cols.loc[
    merged_grouped_4_cols["sub_district"].isin(highlighted_sub_districts)
]

In [ ]:
# g = sns.FacetGrid(merged_grouped_4_cols, col="age_group_10", row="sub_district")
# g.map(sns.histplot, "owner_pop_ratio")

In [ ]:
merged_grouped_4_cols.query("roster >= 2015 & roster <= 2020").head(20)

In [ ]:
corr = (
    pop_count_grouped_4_cols.merge(dog_owner_grouped_4_cols, how="outer")
    .fillna(0)
    .corr(numeric_only=True)
)

# Create a mask for the upper triangle
mask = np.triu(np.ones_like(corr, dtype=bool))

fig, ax = plt.subplots(figsize=(10, 8))

# Create a heatmap plot with a color gradient
sns.heatmap(
    corr.mask(mask),
    cmap="coolwarm",
    annot=True,
    fmt=".2f",
    square=True,
    linewidths=0.5,
    cbar_kws={"shrink": 0.8},
    ax=ax,
)

# Set the title and labels
ax.set_title("Correlation Matrix")

# Adjust the layout
plt.tight_layout()

# Display the plot
plt.show()

In [ ]:
pop_data_filtered.sample().T

#### Zurich Income Dataset
These data contain quantile values of the taxable income of natural persons who are primarily taxable in the city of Zurich. Tax income are in thousand francs (integer).

In [ ]:
# zurich_income_link = "https://data.stadt-zuerich.ch/dataset/fd_median_einkommen_quartier_od1003/download/WIR100OD1003.csv"
zurich_income_link = (
    "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_income.csv"
)
zurich_income_data = pd.read_csv(zurich_income_link)
zurich_income_data.info()
zurich_income_data.columns

In [ ]:
zurich_income_data = hf.sanitize_df_column_names(zurich_income_data)
zurich_income_data.info()
zurich_income_data.columns

In [ ]:
zurich_income_data.sample(5)

In [ ]:
# Extract unique values from 'tax_tariff_long' column and convert to list
tax_tariff_long_de = zurich_income_data.tax_tariff_long.unique().tolist()

# Translate the list to a dictionary using a helper function
tax_tariff_long_translated = translate_list_to_dict(tax_tariff_long_de)

# Display the translated dictionary for verification
display(tax_tariff_long_translated)

# Map the translated dictionary to 'tax_tariff_long' column, creating a new 'tax_status' column
zurich_income_data["tax_status"] = zurich_income_data.tax_tariff_long.map(
    tax_tariff_long_translated
)

# Create new columns with copied data for further processing
zurich_income_data["neighborhood"] = zurich_income_data["quar_lang"].copy()
zurich_income_data["roster"] = zurich_income_data["deadline_date_year"].copy()

# Create 'sub_district' column, ensuring it's a string with leading zeros
zurich_income_data["sub_district"] = (
    zurich_income_data["quar_cd"].copy().astype(str).str.zfill(3)
)

# Create 'district' column by extracting first two characters from 'sub_district'
zurich_income_data["district"] = (
    zurich_income_data["sub_district"].str[:2].astype(int).copy()
)

# Create new columns for income percentiles
zurich_income_data["median_income"] = zurich_income_data["tax_income_p_50"].copy()
zurich_income_data["lower_q_income"] = zurich_income_data["tax_income_p_25"].copy()
zurich_income_data["upper_q_income"] = zurich_income_data["tax_income_p_75"].copy()

# Define a list of columns of interest for the final dataframe
columns_of_interest_income = [
    "neighborhood",
    "roster",
    "district",
    "sub_district",
    "tax_status",
    "median_income",
    "lower_q_income",
    "upper_q_income",
]

# Create a new dataframe 'income_data' with only the columns of interest
income_data = (
    zurich_income_data[columns_of_interest_income].query("roster >= 2015").copy()
)

# Display the structure of 'income_data' dataframe for verification
income_data.info()

In [ ]:
income_data.describe(include="all").T.sort_values(by="unique").fillna("")

In [ ]:
income_data.sample(10)
income_data.to_csv("../data/processed_income_data.csv", index=False)

In [ ]:
income_agg_data_to_merge = (
    income_data
    .groupby(["roster", "sub_district"])["median_income"]
    .median()
    .reset_index()
)


# income_agg
# income_data.query("roster >= 2015")[
#     ["roster", "sub_district", "median_income", "tax_status"]
# ].merge(merged_grouped_4_cols, on=["roster", "sub_district"], how="inner")

merged_3_datasets = pd.merge(
    merged_grouped_4_cols.query("roster >= 2015 & roster <= 2020"),
    income_agg_data_to_merge,
    on=["roster", "sub_district"],
    how="left",
)
merged_3_datasets.head(20)

#### Zurich Household data 

In [ ]:
zurich_household_data_link = "https://data.stadt-zuerich.ch/dataset/bev_hh_haushaltsgroesse_quartier_seit2013_od3806/download/BEV380OD3806.csv"
zurich_household_data_link = (
    "https://storage.googleapis.com/mrprime_dataset/zurich/zurich_household.csv"
)
zurich_household_data = pd.read_csv(zurich_household_data_link)
zurich_household_data.info()

In [ ]:
zurich_household_data = hf.sanitize_df_column_names(zurich_household_data)
zurich_household_data.info()
zurich_household_data.head(5)

In [ ]:
zurich_household_data["neighborhood"] = zurich_household_data["quar_lang"].copy()
zurich_household_data["roster"] = zurich_household_data["key_day_dat_year"].copy()
zurich_household_data["sub_district"] = (
    zurich_household_data["quar_sort"].copy().astype(str).str.zfill(3)
)
zurich_household_data["district"] = (
    zurich_household_data["sub_district"].str[:2].astype(int).copy()
)
zurich_household_data["household_size"] = (
    zurich_household_data["hh_size_sort"].copy().astype(str).str.zfill(2)
)
zurich_household_data["household_count"] = zurich_household_data["number_hh"].copy()
zurich_household_data["resident_count"] = zurich_household_data["number_we"].copy()

# create a dataframe with only the columns of interest
columns_of_interest_household = [
    "neighborhood",
    "roster",
    "district",
    "sub_district",
    "household_size",
    "household_count",
    "resident_count",
]
household_data = zurich_household_data[columns_of_interest_household].query(
    "roster >= 2015 & roster <= 2020"
)

household_data.info()
household_data.head(10)

In [ ]:
household_data.describe(include="all").T.sort_values(by="unique").fillna("")

In [ ]:
# Calculate total residents per sub_district
household_data["total_residents"] = household_data.groupby(["roster", "sub_district"])[
    "resident_count"
].transform("sum")

# Calculate the total households per sub_district
household_data["total_households"] = household_data.groupby(["roster", "sub_district"])[
    "household_count"
].transform("sum")

# Average household size
household_data["avg_household_size"] = (
    household_data["total_residents"] / household_data["total_households"]
)

# Calculate weighted average household size
household_data["resident_portion"] = (
    household_data["resident_count"] / household_data["total_residents"]
)
# household_data

hh_data_to_merge = (
    household_data.groupby(["roster", "sub_district"])[
        ["avg_household_size", "total_households"]
    ]
    .mean()
    .reset_index()
)


In [ ]:
# Save the proprocessed housing data to csv
hh_data_to_merge.to_csv("../data/processed_household_data.csv", index=False)

Reload household dataset that was proprocessed

In [ ]:
# load the processed housing data
hh_data_to_merge = pd.read_csv("../data/processed_household_data.csv")
hh_data_to_merge["sub_district"] = (
    hh_data_to_merge["sub_district"]astype(int).astype(str).str.zfill(3)
)
hh_data_to_merge["total_households"] = hh_data_to_merge["total_households"].astype(int)
hh_data_to_merge.info()
hh_data_to_merge.head()

In [ ]:
merged_4_datasets = pd.merge(
    merged_3_datasets,
    hh_data_to_merge,
    on=["roster", "sub_district"],
    how="left",
)


merged_4_datasets.info()

##### Local Indicators of Spatial Association (LISA)
This LISA plot shows spatial autocorrelation of number of dog owners per sub-district in 2020. Quadrants represent:

1. High-High (HH): "Hot spots" of high dog ownership.
2. Low-Low (LL): "Cold spots" of low dog ownership.
3. High-Low (HL) and Low-High (LH): "Spatial outliers" of contrasting dog ownership.

Areas with p-value < 0.05 are statistically significant and highlighted. This reveals spatial distribution and clustering of dog owners across sub-districts.

In [ ]:
# merged_4_datasets
# neighborhood_gdf

In [ ]:
def calculate_lisa_values(
    df, roster, geodataframe, agg_level, attribute, agg_func="sum"
):
    df_filtered = df.query(f"roster == {roster}")
    df_grouped = df_filtered.groupby(agg_level)[attribute].agg(agg_func)

    n_gdf = geodataframe.set_index(agg_level).sort_index()
    n_gdf = n_gdf.merge(df_grouped, left_index=True, right_index=True, how="left")
    # Find the density of the attribute by dividing by the area
    att_density = f"{attribute}_density"
    n_gdf[att_density] = n_gdf[attribute] / n_gdf["km2"]

    w = lps.weights.Queen.from_dataframe(n_gdf, use_index=False)
    w.transform = "r"

    y = (n_gdf[att_density] - n_gdf[att_density].mean()) / n_gdf[att_density].std()

    li = Moran_Local(y, w)
    plot_local_autocorrelation(li, n_gdf, att_density, p=0.05, aspect_equal=True)
    plt.suptitle(f"Local Moran's I for {att_density} in {roster}")

    return li.Is, n_gdf.index


df_for_spatial_analysis = merged_4_datasets[
    ["roster", "sub_district", "owner_count", "pop_count"]
]
attributes = ["owner_count", "pop_count"]
rosters = df_for_spatial_analysis.roster.unique()

results = {
    f"{attribute}_{roster}": calculate_lisa_values(
        df_for_spatial_analysis, roster, neighborhood_gdf, "sub_district", attribute
    )
    for roster in rosters
    for attribute in attributes
}

lisa_df = pd.DataFrame({key: value[0] for key, value in sorted(results.items())})
lisa_df.index = list(results.values())[0][1]
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(lisa_df, cmap="RdBu_r", ax=ax, center=0)
plt.title(f"Local Moran's I for ")
plt.show()

In [ ]:
merged_4_datasets

In [ ]:
hf.get_line_plots(
    merged_4_datasets.groupby(["roster", "sub_district"])["owner_count"]
    .sum()
    .reset_index(),
    x="roster",
    group_by="sub_district",
    highlight_list=highlighted_sub_districts,
).opts(
    height=800,
    width=600,
    legend_cols=2,
    title="Owner Count by Sub District",
    hooks=[update_xaxis],
)

In [ ]:
from sklearn.linear_model import LinearRegression

pop_owner_agg = merged_4_datasets.groupby(["roster", "sub_district"], as_index=False)[
    ["pop_count", "owner_count"]
].sum()
sns.regplot(x="roster", y="owner_count", data=pop_owner_agg)
plt.title(f"Seaborn Regression Plot | Owner Count")
plt.show()


# Create a linear regression model
model = LinearRegression()

# Reshape the data to fit the model
X = pop_owner_agg["roster"].values.reshape(-1, 1)
y = pop_owner_agg["owner_count"].values

# Fit the model
model.fit(X, y)

# fit trend
y_pred = model.predict(X)

# Get the coefficients
slope = model.coef_[0]
intercept = model.intercept_

In [ ]:
# plot the regression line and the actual data
pop_owner_agg.plot.scatter(x="roster", y="owner_count")
plt.plot(X, y_pred, color="red")
plt.xlabel("Year")
plt.ylabel("Owner Count")
plt.title("Owner Count Trend")
plt.show()

In [ ]:
# Aggregate the pop_count and owner_count by roster and sub_district
pop_owner_agg = merged_4_datasets.groupby(["roster", "sub_district"], as_index=False)[
    ["pop_count", "owner_count"]
].sum()
pop_owner_agg["roster"] = pd.to_datetime(
    pop_owner_agg["roster"].astype("string"), format="%Y"
)
pop_owner_agg["roster"]

In [ ]:
df_for_spatial_analysis = merged_4_datasets[["roster", "sub_district", "owner_count"]]


lisa_df = pd.DataFrame()

for roster in [2020]:
    df = df_for_spatial_analysis.query(f"roster == {roster}")
    df = df.groupby("sub_district")["owner_count"].sum()
    # df = df.set_index("sub_district")
    n_gdf = neighborhood_gdf.set_index("sub_district")
    n_gdf = n_gdf.merge(df, left_index=True, right_index=True, how="left")

    w = lps.weights.Queen.from_dataframe(n_gdf, use_index=False)
    w.transform = "r"

    # Standardize the attribute
    y = (n_gdf["owner_count"] - n_gdf["owner_count"].mean()) / n_gdf[
        "owner_count"
    ].std()

    li = Moran_Local(y, w)
    plot_local_autocorrelation(li, n_gdf, "owner_count", p=0.05, aspect_equal=True)

    # Save the LISA values in the DataFrame
    lisa_df[roster] = li.Is

    # Set the index of the DataFrame to be the sub-districts
    lisa_df.index = n_gdf.index


lisa_df

In [ ]:
# dog_data["age_group_10"] = pd.Categorical(dog_data["age_group_10"], ordered=True)


# dog_data["age_group_10"].cat.categories


# dog_data["roster"] = dog_data["roster"].astype(str)


# dog_data["roster"] = pd.Categorical(dog_data["roster"], ordered=True)


# dog_data["roster"].cat.categories

In [ ]:
dog_data.sample()
zurich_income_data.loc[zurich_income_data["roster"] > 2014][
    ["tax_status", "tax_income_p_50", "district", "sub_district", "roster"]
]
tax_status_button = pnw.RadioButtonGroup(
    name="Tax Status",
    options=list(zurich_income_data["tax_status"].unique()),
    value="Basic tariff",
    button_style="outline",
    button_type="success",
)
roster_button = pnw.RadioButtonGroup(
    value=2018,
    options=list(range(2015, 2021)),
    button_style="outline",
    button_type="success",
)


@pn.depends(roster_button.param.value, tax_status_button.param.value)
def get_income_polygon(roster, tax_status):
    df = zurich_income_data.loc[
        (zurich_income_data["roster"] == roster)
        & (zurich_income_data["tax_status"] == tax_status)
    ][["roster", "tax_status", "tax_income_p_50", "sub_district"]]

    income_gdf = neighborhood_gdf.merge(df, on="sub_district")
    # income_gdf = income_gdf.drop(columns=["km2"])

    income_cmap = list(sns.color_palette("light:" + "#008800", n_colors=6).as_hex())
    return gv.Polygons(income_gdf).opts(
        **poly_opts,
        color="tax_income_p_50",
        # fill_alpha=0,
        cmap=income_cmap,
        clim=(30, 100),
        colorbar=True,
        tools=["hover", "tap", "box_select"],
        title=f"{tax_status.title()} | {roster}",
    )


income_chloropleth = pn.pane.HoloViews(get_income_polygon)

pn.Column(tax_status_button, roster_button, get_income_polygon)

In [ ]:
unique_breeds = dog_data["standard"].unique()
num_unique_breeds = len(unique_breeds)

# Repeat the colormap to cover all unique breeds
repeated_cmap = list(cc.glasbey_dark) * \
    (num_unique_breeds // len(cc.glasbey_dark) + 1)

# Explicit mapping for the color to use for each standard breed
explicit_mapping = {breed: repeated_cmap[i]
                    for i, breed in enumerate(unique_breeds)}
my_colors = hv.Cycle(list(explicit_mapping.values()))


# colormaps for the the gender
boy_cmap = list(sns.color_palette("light:#00008b", n_colors=6).as_hex())


girl_cmap = list(sns.color_palette("light:#8b008b", n_colors=6).as_hex())

In [ ]:
neighborhood_gdf["km2"] = neighborhood_gdf.to_crs(
    ccrs.GOOGLE_MERCATOR).area / 10**6

neighborhood_gdf["km2"] = neighborhood_gdf["km2"].round(2)

districts_gdf["km2"] = districts_gdf.to_crs(ccrs.GOOGLE_MERCATOR).area / 10**6

districts_gdf["km2"] = districts_gdf["km2"].round(2)

districts_gdf

In [ ]:
hv.opts.defaults(
    hv.opts.HeatMap(
        tools=["hover", "tap", "box_select"],
        active_tools=["box_zoom"],
        width=800,
        height=500,
        ylabel="",
        line_color="white",
        line_width=2,
        toolbar="above",
        selection_line_color="red",
        nonselection_alpha=0.9,
        nonselection_line_color="white",
    )
)

poly_opts = dict(
    width=800,
    height=500,
    line_width=2,
    xaxis=None,
    yaxis=None,
    aspect="equal",
    # cmap=explicit_mapping,
    # projection=ccrs.GOOGLE_MERCATOR,
)
bar_opts = dict(
    invert_axes=True,
    width=800,
    height=500,
    tools=["hover", "tap", "box_select"],
    xaxis=None,
    ylabel="",
    xlabel="",
    cmap=explicit_mapping,
    show_legend=False,
    active_tools=["box_zoom"],
)

In [ ]:
tools_opts = dict(active_tools=["box_zoom"], toolbar="above")
# bar plot of the owner age distribution
owner_age_groups_all = (
    dog_data.groupby(["age_group_10", "roster"])
    .size()
    .unstack()
    .rename(columns={0: "Count"})
    .hvplot.bar(
        xlabel="",
        rot=90,
        legend=True,
        tools=["hover", "box_select"],
        title="Owners Age distribution",
        muted_line_alpha=0.01,
    )
    .opts(**tools_opts)
)
# kde plot of the dog age distribution
dog_age_all = dog_data.hvplot.kde(
    y="dog_age",
    by="roster",
    xlim=(0, 21),
    ylim=(0, 0.1),
    xlabel="",
    legend=True,
    tools=["hover", "box_select"],
    title="Dog Age Density distribution",
    muted_alpha=0.01,
).opts(**tools_opts)
# step plot of the dog age distribution
dog_age_all_step = (
    dog_data.groupby(["dog_age", "roster"])
    .size()
    .hvplot.step(
        x="dog_age",
        by="roster",
        xlim=(0, 21),
        title="Dog Age distribution",
    )
    .opts(**tools_opts)
)
# panel card with the 3 plots
all_ages_card = pn.Card(
    # add spacer
    pn.Spacer(height=70),
    dog_age_all_step,
    dog_age_all,
    owner_age_groups_all,
    title="Age Distribution for ALL Dogs and Owners",
    styles={"background": "gainsboro"},
)

In [ ]:
pn.state.kill_all_servers()
all_ages_card

In [ ]:
def plot_age_range_stats(df, breed, bw, is_pure_breed):
    """Function to plot the age_range stats for the breed owners and the dog age distribution"""
    if is_pure_breed:
        breed_df = df.query(
            f'standard.str.contains("{breed}", case=False, regex=True, na=False) & is_pure_breed == True'
        )
    else:
        breed_df = df.query(
            f'standard.str.contains("{breed}", case=False, regex=True, na=False)'
        )
    # plot a step plot of the dog_age
    step = (
        breed_df.groupby(["dog_age", "roster"])
        .size()
        .unstack()
        .hvplot.step(
            title=f"Dog Age distribution | {breed.title()}",
            shared_axes=False,
            muted_alpha=0.1,
        )
    )

    # plot age_range of breed_df kde plot
    kde = breed_df.hvplot.kde(
        y="dog_age",
        by="roster",
        bandwidth=bw,
        xlim=(0, 20),
        ylim=(0, None),
        xlabel="",
        muted_alpha=0.01,
        title=f"Dog Age density distribution | {breed.title()}",
    )
    # plot age_range of breed_df bar plot

    bar = (
        breed_df.groupby(["age_group_10", "roster"])
        .size()
        .unstack()
        .hvplot.bar(
            rot=90,
            legend=True,
            tools=["hover", "box_select"],
            title=f"Owner Age distribution | {breed.title()} ",
            shared_axes=False,
            xlabel="",
        )
        .opts(active_tools=["box_select"])
    )
    return (step + kde + bar).cols(1)


# create widgets for the breed, bandwidth
breed_selector = pnw.Select(name="Breed", options=topn, value="french bulldog")
top_n_slider = pnw.IntSlider(
    name="Top N", start=1, end=30, step=1, value=10, width=200)
bandwidth_slider = pnw.FloatSlider(
    name="Bandwidth", start=0.1, end=2, step=0.1, value=0.5, width=200
)
pure_breed_checkbox = pnw.Checkbox(name="Pure Breed", value=True, width=200)
is_male_owner_checkbox = pnw.Checkbox(
    name="Male Dog Owner", value=True, width=200)
roster_slider = pnw.DiscreteSlider(
    options=["2015", "2016", "2017", "2018",
             "2019", "2020", "2021", "2022", "2023"],
    name="Roster",
    width=200,
)
show_labels_checkbox = pnw.Checkbox(
    name="Show Annotations", value=True, width=200)


# create a dynamic map

dynamic_age_plot = pn.bind(
    plot_age_range_stats,
    df=dog_data,
    breed=breed_selector,
    bw=bandwidth_slider,
    is_pure_breed=pure_breed_checkbox,
)
widget_controls = pn.Row(
    pure_breed_checkbox,
    breed_selector,
    bandwidth_slider,
    styles={"background": "mintcream"},
)
dynamic_age_card = pn.Card(
    widget_controls,
    dynamic_age_plot,
    title="Age Distribution of Dog and Owners by Breeds",
    styles={"background": "mintcream"},
)

In [ ]:
# pn.Column(dynamic_age_card).show()
districts_gdf = districts_gdf.dissolve(by="district")
districts_gdf
# dog_data
# dynamic_age_card

In [ ]:
filtered_dog_data

In [ ]:
pn.state.kill_all_servers()


# get the count of the french bulldog in each district for each roster and plot it
# plot it on a dynamic map with a slider for the roster
def get_breed_count(breed, roster):
    df = filtered_dog_data[
        (filtered_dog_data["standard"] == breed)
        & (filtered_dog_data["roster"] == roster)
    ][["district", "standard", "roster"]]
    df = df.groupby("district").size().reset_index(name="count")
    df = df.set_index("district")

    breed_gdf = districts_gdf.merge(df, left_index=True, right_index=True, how="left")
    # breed_gdf = breed_gdf.drop(columns=["desc", "km2"])
    breed_color = explicit_mapping[breed]
    # Start from white and go to the breed color
    breed_cmap = list(sns.color_palette("light:" + breed_color, n_colors=6).as_hex())
    return gv.Polygons(breed_gdf).opts(
        **poly_opts,
        color="count",
        # fill_alpha=0,
        cmap=breed_cmap,
        clim=(0, 50),
        colorbar=True,
        tools=["hover", "tap", "box_select"],
        title=f"{breed.title()} | {roster}",
    )


standard_selector = pnw.Select(name="Breed", options=topn, value="french bulldog")
# create the dynamic map
breed_chloro = gv.DynamicMap(
    pn.bind(get_breed_count, breed=standard_selector, roster=roster_button)
)

In [ ]:
basemap = gv.tile_sources.OSM().opts(alpha=0.5, bgcolor="black")
pn.Column(standard_selector, pn.Row(roster_button, breed_chloro)).show()
# breed_chloro

In [ ]:
pn.state.kill_all_servers()


neighborhood_poly = gv.Polygons(neighborhood_gdf).opts(
    tools=["hover", "tap"],
    **poly_opts,
    line_color="crimson",
    # fill_color="lightgray",
    # fill_alpha=0,
    alpha=0.2,
)
neighborhood_poly

pn.pane.HoloViews(breed_chloro)

In [ ]:
neighborhood_poly.data
breed_chloro.data

In [ ]:
districts_gdf = (
    neighborhood_gdf.dissolve(by="district")
    .drop(columns=["neighborhood", "sub_district"])
    .reset_index()
)

districts_poly = gv.Polygons(districts_gdf).opts(
    **poly_opts,
    color_index=None,
)

# (district_poly * neighborhood_poly).opts(title="Zurich Districts and Neighborhoods")

In [ ]:
zurich_desc_path = "../data/zurich_districts.csv"
zurich_desc = pd.read_csv(zurich_desc_path)
# zurich_desc["district"] = zurich_desc["district"].astype(str).str.zfill(2)
districts_gdf["district"] = districts_gdf["district"].astype(int)

districts_gdf = districts_gdf.merge(
    zurich_desc[["district", "desc", "district_name"]], on="district", how="left"
)
# districts_gdf.set_index("district", inplace=True)
districts_poly = gv.Polygons(districts_gdf)

In [ ]:
districts_gdf = districts_gdf.dissolve(by="district")
districts_poly = gv.Polygons(districts_gdf)

In [ ]:
# create a stream which selects a district from the map
select_district = streams.Selection1D(source=districts_poly)


def display_info(index):
    if not index:
        return pn.pane.Markdown("No district selected")
    else:
        selected_district = (
            districts_poly.iloc[index[0]]
            .data[["district", "district_name", "desc"]]
            .drop_duplicates()
        )
        dname = selected_district["district_name"].values[0]
        dnum = selected_district["district"].values[0]
        ddesc = selected_district["desc"].values[0]
        return pn.pane.Markdown(
            f"#### {dnum}\n ### {dname}\n {ddesc}",
        )


def display_wordcloud(index, breed):
    breed_color = explicit_mapping[breed]
    if len(index) == 0:
        text = "district select on map"
        wordcloud = WordCloud(width=800, height=500, background_color="white").generate(
            text
        )
        return hv.RGB(np.array(wordcloud))
    else:
        selected_district = (
            districts_poly.iloc[index[0]]
            .data[["district", "district_name", "desc"]]
            .drop_duplicates()
        )
        dname = selected_district["district_name"].values[0]
        dnum = selected_district["district"].values[0]
        ddesc = selected_district["desc"].values[0]
        text = f"{dnum} {dname} {ddesc}"

        polygon = districts_poly.iloc[index[0]].data["geometry"].iloc[0]

        # Get the bounding box of the polygon
        minx, miny, maxx, maxy = polygon.bounds

        # Calculate the width and height of the bounding box
        width = maxx - minx
        height = maxy - miny

        # Create a new image with the same aspect ratio as the bounding box
        image_width = 800
        image_height = int(image_width * height / width)
        test = Image.new("1", (image_width, image_height), 0)

        # Convert the coordinates to a numpy array
        coords = np.array(list(polygon.exterior.coords))
        coords -= [minx, miny]
        coords *= [image_width / width, image_height / height]
        coords[:, 1] = image_height - coords[:, 1]
        # Convert the coordinates back to a list of tuples
        scaled_coords = list(map(tuple, coords))

        # Draw the scaled polygon onto the image
        ImageDraw.Draw(test).polygon(scaled_coords, outline=1, fill=1)

        wordcloud = WordCloud(
            mask=~np.array(test) * 255,
            color_func=lambda *args, **kwargs: breed_color,
            include_numbers=True,
            margin=20,
            contour_color=breed_color,
            contour_width=5,
            width=800,
            height=500,
            background_color="white",
        ).generate(text)
        return hv.RGB(np.array(wordcloud)).opts(
            width=800,
            height=500,
            tools=["box_zoom"],
            active_tools=["box_zoom"],
            xaxis=None,
            yaxis=None,
            # toolbar=None,
        )


layout = pn.Column(
    pn.bind(display_wordcloud, select_district.param.index, breed=breed_selector),
    districts_poly * breed_chloro,
    pn.bind(display_info, select_district.param.index),
    width=800,
)

breed_chloro_card = pn.Card(
    roster_slider, layout, title="Chloropleth of Selected Breed"
)

In [ ]:
dynamic_age_card

In [ ]:
pn.Row(
    dynamic_age_card,
    breed_chloro_card,
    all_ages_card,
    sizing_mode="stretch_width",
)

Gender Heatmap


In [ ]:
# Create a Tap stream linked to the HeatMap
breed_tap = streams.Tap(source=None)


@pn.depends(roster_slider.param.value, is_male_owner_checkbox.param.value)
def get_gender_roster_df(roster, gender):
    return dog_data.loc[
        (dog_data["is_male_owner"] == gender) & (dog_data["roster"] == roster)
    ]


@pn.depends(
    roster_slider.param.value,
    is_male_owner_checkbox.param.value,
    top_n_slider.param.value,
)
def get_top_n_gender_breeds(roster, gender, top_n):
    gender_roster_df = get_gender_roster_df(roster=roster, gender=gender)
    return gender_roster_df["standard"].value_counts().head(top_n).index.tolist()


@pn.depends(
    roster_slider.param.value,
    is_male_owner_checkbox.param.value,
    top_n_slider.param.value,
)
def get_gender_heatmap(roster, gender, top_n):
    gender_roster_df = get_gender_roster_df(roster=roster, gender=gender)
    topn_gender_breeds = get_top_n_gender_breeds(
        roster=roster, gender=gender, top_n=top_n
    )

    top_gender_breeds_df = (
        gender_roster_df.loc[gender_roster_df["standard"].isin(topn_gender_breeds)]
        .groupby(["standard", "district"])
        .size()
        .fillna(0)
        .reset_index(name="count")
    )
    sex = "Male" if gender else "Female"
    top_gender_breeds_heatmap = hv.HeatMap(
        top_gender_breeds_df, ["district", "standard"], "count"
    ).redim(standard="gender_standard")
    top_gender_breeds_heatmap.opts(
        height=(33 * top_n) + 50,
        cmap=boy_cmap if gender else girl_cmap,
        colorbar=True,
        active_tools=["box_zoom"],
        title=f"Top {top_n} breeds | {roster} | {sex} Owners",
        clim=(0, 50),
    )
    breed_tap.source = top_gender_breeds_heatmap

    return top_gender_breeds_heatmap


dynamic_gender_heatmap_panel = pn.pane.HoloViews(get_gender_heatmap)

# pn.Column(
#     roster_slider,
#     holder_gender_checkbox,
#     top_n_slider,
#     dynamic_gender_heatmap_panel,
# ).show()

In [ ]:
@pn.depends(
    breed_tap.param.x,
    breed_tap.param.y,
    roster_slider.param.value,
    is_male_owner_checkbox.param.value,
)
def get_breed_chloropleth(x, y, roster, gender):
    if x is None or y is None:
        return gv.Polygons(districts_gdf).opts(**poly_opts, title="Select a cell")
    else:
        data = get_gender_roster_df(roster=roster, gender=gender)
        data = data.loc[data["standard"] == y]
        data = data.groupby("district").size().reset_index(name="count")
        data = data.set_index("district")

        breed_gdf = districts_gdf.merge(
            data, left_index=True, right_index=True, how="left"
        )
        # breed_gdf = breed_gdf.drop(columns=["desc", "km2"])
        breed_gdf.fillna(0, inplace=True)
        breed_color = explicit_mapping[y]
        breed_cmap = list(
            sns.color_palette("light:" + breed_color, n_colors=6).as_hex()
        )
        sex = "Male" if gender else "Female"

        return gv.Polygons(breed_gdf).opts(
            **poly_opts,
            color="count",
            cmap=breed_cmap,
            clim=(0, 50),
            colorbar=True,
            line_color="darkgray",
            tools=["hover", "tap", "box_select"],
            title=f"{y.title()} | {roster} | {sex} Owners",
        )


breed_chloropleth = pn.pane.HoloViews(get_breed_chloropleth)


# Combine the heatmap and the text display into a layout
layout = pn.Column(
    roster_slider,
    is_male_owner_checkbox,
    top_n_slider,
    dynamic_gender_heatmap_panel,
    breed_chloropleth,
)

# layout.show()

In [ ]:
`# Define a Tap stream linked to the owner age group`
owner_tap = streams.Tap(source=None, x=1, y=30)


@pn.depends(roster_slider.param.value, is_male_owner_checkbox.param.value)
def get_age_heatmap(roster, gender):
    data = get_gender_roster_df(roster=roster, gender=gender)
    data_grouped = (
        data.groupby(["district", "age_group_10"])["owner_id"]
        .nunique()
        .reset_index()
        .rename(columns={"owner_id": "count"})
    )
    sex = "Male" if gender else "Female"
    heatmap = hv.HeatMap(data_grouped, ["district", "age_group_10"], "count")

    heatmap.opts(
        # **heatmap_opts,
        cmap=boy_cmap if gender else girl_cmap,
        ylim=(0, 100),
        xlim=(0, 13),
        colorbar=True,
        active_tools=["box_zoom"],
        tools=["hover", "tap", "box_select"],
        title=f"{sex} Dog Owners | {roster} | by Age Group vs District",
    )
    owner_tap.source = heatmap
    return heatmap


age_group_panel = pn.pane.HoloViews(get_age_heatmap)

In [ ]:
dog_data["age_group_10"].cat.categories

In [ ]:
import math

pn.state.kill_all_servers()
# Bar Plot which shows the top 10 breeds for each district and age group as it is selected in the heatmap

bar_plots_opts = dict(
    height=500,
    width=800,
    invert_axes=True,
    cmap=explicit_mapping,
    show_legend=False,
    xlabel="",
    fontscale=1.2,
)


@pn.depends(
    owner_tap.param.x,
    owner_tap.param.y,
    roster_slider.param.value,
    is_male_owner_checkbox.param.value,
)
def update_barplot(x, y, roster, gender):
    data = get_gender_roster_df(roster=roster, gender=gender)
    if x is None or y is None:
        bar_data = (
            data["standard"]
            .value_counts()
            .head(10)
            .rename("count")
            .reset_index()
            .rename(columns={"index": "standard"})
        )
        return hv.Bars(bar_data, kdims=["standard"], vdims="count").opts(
            **bar_plots_opts,
            color="standard",
            title=f"Top 10 Breeds",
            tools=["hover"],
            active_tools=["box_zoom"],
        )
    x = math.ceil(x - 0.5)
    y = math.ceil(y / 10) * 10
    bar_data = (
        data.loc[(data["district"] == x) & (data["age_group_10"] == y)]["standard"]
        .value_counts()
        .head(10)
        .reset_index()
    )
    bar_data.columns = ["standard", "count"]
    if len(bar_data) == 0:
        return hv.Bars([], "standard", "count").opts(
            **bar_plots_opts,
            title=f"No Breeds for Age-group:{y} | Districts:{x}",
            active_tools=["box_zoom"],
        )

    return hv.Bars(bar_data, "standard", "count").opts(
        **bar_plots_opts,
        color="standard",
        title=f"Top {min(10,len(bar_data))} Popular Breeds | Age-group:{y} | Districts:{x}",
        active_tools=["box_zoom"],
    )


update_barplot_panel = pn.pane.HoloViews(update_barplot)
pn.Column(
    roster_slider,
    is_male_owner_checkbox,
    age_group_panel,
    update_barplot_panel,
)